<a href="https://colab.research.google.com/github/vaniamv/dataprocessing/blob/main/spark_streaming/challenges/final_challenges.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setting up PySpark

In [ ]:
%pip install pyspark

# Context
Message events are coming from platform message broker (kafka, pubsub, kinesis...).
You need to process the data according to the requirements.

Message schema:
- timestamp
- value
- event_type
- message_id
- country_id
- user_id



# Challenge 1

Step 1
- Change exising producer
	- Change parquet location to "/content/lake/bronze/messages/data"
	- Add checkpoint (/content/lake/bronze/messages/checkpoint)
	- Delete /content/lake/bronze/messages and reprocess data
	- For reprocessing, run the streaming for at least 1 minute, then stop it

Step 2
- Implement new stream job to read from messages in bronze layer and split result in two locations
	- "messages_corrupted"
		- logic: event_status is null, empty or equal to "NONE"
		- extra logic: add country name by joining message with countries dataset
		- partition by "date" -extract it from timestamp
		- location: /content/lake/silver/messages_corrupted/data

	- "messages"
		- logic: not corrupted data
		- extra logic: add country name by joining message with countries dataset
		- partition by "date" -extract it from timestamp
		- location: /content/lake/silver/messages/data

	- technical requirements
		- add checkpint (choose location)
		- use StructSchema
		- Set trigger interval to 5 seconds
		- run streaming for at least 20 seconds, then stop it

	- alternatives
		- implementing single streaming job with foreach/- foreachBatch logic to write into two locations
		- implementing two streaming jobs, one for messages and another for messages_corrupted
		- (paying attention on the paths and checkpoints)


  - Check results:
    - results from messages in bronze layer should match with the sum of messages+messages_corrupted in the silver layer

In [1]:
%pip install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 15.8 MB/s eta 0:00:00


# Producer

In [132]:
import pyspark.sql.functions as F
from pyspark.sql import DataFrame
from faker import Faker
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('Test streaming').getOrCreate()
sc = spark.sparkContext

fake = Faker()
messages = [fake.uuid4() for _ in range(50)]

def enrich_data(df, messages=messages):
  fake = Faker()
  new_columns = {
      'event_type': F.lit(fake.random_element(elements=('OPEN', 'RECEIVED', 'SENT', 'CREATED', 'CLICKED', '', 'NONE'))),
      'message_id': F.lit(fake.random_element(elements=messages)),
      'channel': F.lit(fake.random_element(elements=('CHAT', 'EMAIL', 'SMS', 'PUSH', 'OTHER'))),
      'country_id': F.lit(fake.random_int(min=2000, max=2015)),
      'user_id': F.lit(fake.random_int(min=1000, max=1050)),
  }
  df = df.withColumns(new_columns)
  return df

def insert_messages(df: DataFrame, batch_id):
  enrich = enrich_data(df)
  enrich.write.mode("append").format("parquet").save("content/lake/bronze/messages/data")

# read stream
df_stream = spark.readStream.format("rate").option("rowsPerSecond", 1).load()

# write stream
query = (df_stream.writeStream
.outputMode('append')
.option('checkpointLocation', 'content/lake/bronze/checkpoint')
.trigger(processingTime='1 seconds')
.foreachBatch(insert_messages)
.start()
)

query.awaitTermination(60)


False

In [135]:
query.isActive

True

In [150]:
query.stop()

In [131]:
!rm -rf content

In [136]:
df = spark.read.format("parquet").load("content/lake/bronze/messages/*")
df.show()

+--------------------+-----+----------+--------------------+-------+----------+-------+
|           timestamp|value|event_type|          message_id|channel|country_id|user_id|
+--------------------+-----+----------+--------------------+-------+----------+-------+
|2024-12-10 23:41:...|   25|  RECEIVED|85b421d2-7b88-407...|  EMAIL|      2014|   1010|
|2024-12-10 23:41:...|   51|  RECEIVED|4e0216fc-a021-49f...|  EMAIL|      2012|   1038|
|2024-12-10 23:41:...|   21|   CLICKED|353cfd0b-af7b-43a...|  EMAIL|      2011|   1006|
|2024-12-10 23:41:...|   63|   CLICKED|66b5ddff-f349-444...|  EMAIL|      2008|   1003|
|2024-12-10 23:41:...|   35|  RECEIVED|0e8296fe-10c2-471...|   CHAT|      2012|   1016|
|2024-12-10 23:40:...|   15|   CREATED|af8a9603-99ae-47d...|  EMAIL|      2015|   1046|
|2024-12-10 23:41:...|   65|  RECEIVED|2f06b118-33d9-4d0...|   PUSH|      2007|   1022|
|2024-12-10 23:40:...|    1|   CLICKED|9f249aa4-f088-4c0...|  OTHER|      2009|   1044|
|2024-12-10 23:41:...|   20|   C

In [82]:
df.schema

StructType([StructField('timestamp', TimestampType(), True), StructField('value', LongType(), True), StructField('event_type', StringType(), True), StructField('message_id', StringType(), True), StructField('channel', StringType(), True), StructField('country_id', IntegerType(), True), StructField('user_id', IntegerType(), True)])

In [137]:
df.count()

67

# Additional datasets

In [138]:
countries = [
    {"country_id": 2000, "country": "Brazil"},
    {"country_id": 2001, "country": "Portugal"},
    {"country_id": 2002, "country": "Spain"},
    {"country_id": 2003, "country": "Germany"},
    {"country_id": 2004, "country": "France"},
    {"country_id": 2005, "country": "Italy"},
    {"country_id": 2006, "country": "United Kingdom"},
    {"country_id": 2007, "country": "United States"},
    {"country_id": 2008, "country": "Canada"},
    {"country_id": 2009, "country": "Australia"},
    {"country_id": 2010, "country": "Japan"},
    {"country_id": 2011, "country": "China"},
    {"country_id": 2012, "country": "India"},
    {"country_id": 2013, "country": "South Korea"},
    {"country_id": 2014, "country": "Russia"},
    {"country_id": 2015, "country": "Argentina"}
]

countries = spark.createDataFrame(countries)

# Streaming Messages x Messages Corrupted

In [139]:
df.select('event_type').distinct().collect()


[Row(event_type='CLICKED'),
 Row(event_type='CREATED'),
 Row(event_type='RECEIVED'),
 Row(event_type='NONE'),
 Row(event_type='OPEN'),
 Row(event_type='SENT'),
 Row(event_type='')]

In [116]:
!rm -rf content/lake/silver

In [140]:
# TODO
from pyspark.sql.functions import col, lit

corrupted_condition = (col("event_type").isNull() |
                       (col("event_type") == "") |
                       (col("event_type") == "NONE"))

In [141]:
from pyspark.sql.types import StructType, StructField, StringType, LongType, IntegerType, TimestampType


In [142]:
def insert_messages_silver(df: DataFrame, batch_id):
  corrupted_df = df.filter(corrupted_condition)
  corrupted_df = corrupted_df.join(countries, on="country_id", how="left").drop("country_id")
  corrupted_df.write.mode("append").format("parquet").partitionBy("date").save("content/lake/silver/messages_corrupted/data")
  non_corrupted_df = df.filter(~corrupted_condition)
  non_corrupted_df = non_corrupted_df.join(countries, on="country_id", how="left").drop("country_id")
  non_corrupted_df.write.mode("append").format("parquet").partitionBy("date").save("content/lake/silver/messages/data")

schema = StructType([
    StructField('timestamp', TimestampType(), True),
    StructField('value', LongType(), True),
    StructField('event_type', StringType(), True),
    StructField('message_id', StringType(), True),
    StructField('channel', StringType(), True),
    StructField('country_id', IntegerType(), True),
    StructField('user_id', IntegerType(), True)
    ])

# Read the streaming data with the specified schema
messages_df = spark.readStream.format("parquet").schema(schema).load("content/lake/bronze/messages/data")

#extract date from timestamp
messages_df = messages_df.withColumn("date", F.to_date("timestamp"))

query_1 = (messages_df.writeStream
.outputMode('append')
.option('checkpointLocation', 'content/lake/silver/checkpoint')
.trigger(processingTime='5 seconds')
.foreachBatch(insert_messages_silver)
.start()
)

query.awaitTermination(20)


False

In [148]:
query_1.stop()

In [143]:
query_1.isActive

True

In [144]:
df_silver = spark.read.format("parquet").load("content/lake/silver/messages/data/*")
df_silver.show()

+----------+--------------------+-----+----------+--------------------+-------+-------+---------+
|country_id|           timestamp|value|event_type|          message_id|channel|user_id|  country|
+----------+--------------------+-----+----------+--------------------+-------+-------+---------+
|      2009|2024-12-10 23:40:...|    1|   CLICKED|9f249aa4-f088-4c0...|  OTHER|   1044|Australia|
|      2009|2024-12-10 23:41:...|   54|  RECEIVED|ac5e720d-1a42-4fa...|   CHAT|   1041|Australia|
|      2009|2024-12-10 23:41:...|   71|      OPEN|8b4e4747-4b15-472...|  OTHER|   1031|Australia|
|      2009|2024-12-10 23:41:...|   53|      SENT|002b30c5-43f3-400...|   PUSH|   1036|Australia|
|      2009|2024-12-10 23:40:...|    7|      OPEN|af8a9603-99ae-47d...|   PUSH|   1038|Australia|
|      2009|2024-12-10 23:41:...|   69|      SENT|71144736-15db-41c...|    SMS|   1030|Australia|
|      2010|2024-12-10 23:41:...|   74|   CLICKED|dcb25159-8a41-49f...|  OTHER|   1018|    Japan|
|      2010|2024-12-

In [146]:
df_silver_corrupted = spark.read.format("parquet").load("content/lake/silver/messages_corrupted/data/*")
df_silver_corrupted.show()

+----------+--------------------+-----+----------+--------------------+-------+-------+--------------+
|country_id|           timestamp|value|event_type|          message_id|channel|user_id|       country|
+----------+--------------------+-----+----------+--------------------+-------+-------+--------------+
|      2006|2024-12-10 23:41:...|   68|      NONE|4e0216fc-a021-49f...|    SMS|   1031|United Kingdom|
|      2006|2024-12-10 23:41:...|   73|          |9f249aa4-f088-4c0...|  EMAIL|   1027|United Kingdom|
|      2006|2024-12-10 23:40:...|   13|          |353cfd0b-af7b-43a...|   CHAT|   1027|United Kingdom|
|      2001|2024-12-10 23:42:...|  116|          |3778bfec-81ea-4be...|   CHAT|   1021|      Portugal|
|      2001|2024-12-10 23:42:...|  118|          |3778bfec-81ea-4be...|   CHAT|   1021|      Portugal|
|      2001|2024-12-10 23:42:...|  117|          |3778bfec-81ea-4be...|   CHAT|   1021|      Portugal|
|      2004|2024-12-10 23:42:...|  109|          |ea0e44c3-56f0-465...|  

## Checking data

In [ ]:
# TODO

# Challenge 2

- Run business report
- But first, there is a bug in the system which is causing some duplicated messages, we need to exclude these lines from the report

- removing duplicates logic:
  - Identify possible duplicates on message_id, event_type and channel
  - in case of duplicates, consider only the first message (occurrence by timestamp)
  - Ex:
    In table below, the correct message to consider is the second line

```
    message_id | channel | event_type | timestamp
    123        | CHAT    | CREATED    | 10:10:01
    123        | CHAT    | CREATED    | 07:56:45 (first occurrence)
    123        | CHAT    | CREATED    | 08:13:33
```

- After cleaning the data we're able to create the busines report

In [151]:
# dedup data
from pyspark.sql import functions as F
from pyspark.sql.window import Window
df = spark.read.format("parquet").load("content/lake/silver/messages")
dedup = df.withColumn("row_number", F.row_number().over(Window.partitionBy("message_id", "event_type", "channel").orderBy("timestamp"))).filter("row_number = 1").drop("row_number")

In [160]:
dedup.show()

+----------+--------------------+-----+----------+--------------------+-------+-------+-------------+----------+
|country_id|           timestamp|value|event_type|          message_id|channel|user_id|      country|      date|
+----------+--------------------+-----+----------+--------------------+-------+-------+-------------+----------+
|      2011|2024-12-10 23:42:...|   85|   CLICKED|002b30c5-43f3-400...|    SMS|   1027|        China|2024-12-10|
|      2002|2024-12-10 23:41:...|   39|      SENT|002b30c5-43f3-400...|  EMAIL|   1008|        Spain|2024-12-10|
|      2009|2024-12-10 23:41:...|   53|      SENT|002b30c5-43f3-400...|   PUSH|   1036|    Australia|2024-12-10|
|      2013|2024-12-10 23:42:...|  132|   CREATED|09063092-27f1-443...|   PUSH|   1021|  South Korea|2024-12-10|
|      2000|2024-12-10 23:42:...|   84|  RECEIVED|09063092-27f1-443...|   PUSH|   1020|       Brazil|2024-12-10|
|      2012|2024-12-10 23:41:...|   35|  RECEIVED|0e8296fe-10c2-471...|   CHAT|   1016|        I

### Report 1
  - Aggregate data by date, event_type and channel
  - Count number of messages
  - pivot event_type from rows into columns
  - schema expected:
  
```
|      date|channel|CLICKED|CREATED|OPEN|RECEIVED|SENT|
+----------+-------+-------+-------+----+--------+----+
|2024-12-03|    SMS|      4|      4|   1|       1|   5|
|2024-12-03|   CHAT|      3|      7|   5|       8|   4|
|2024-12-03|   PUSH|   NULL|      3|   4|       3|   4|
```

In [159]:
# report 1
# TODO
df.groupBy("date", "channel").pivot("event_type").count().show()

+----------+-------+-------+-------+----+--------+----+
|      date|channel|CLICKED|CREATED|OPEN|RECEIVED|SENT|
+----------+-------+-------+-------+----+--------+----+
|2024-12-10|  OTHER|      3|      5|   3|       1|   1|
|2024-12-10|  EMAIL|      4|      2|   2|       4|   5|
|2024-12-10|    SMS|      4|      4|   1|       2|   4|
|2024-12-10|   CHAT|      5|      4|   3|       2|   5|
|2024-12-10|   PUSH|      3|      3|   4|       3|  11|
+----------+-------+-------+-------+----+--------+----+



## Report 2

- Identify the most active users by channel (sorted by number of iterations)
- schema expected:

```
+-------+----------+----+-----+-----+----+---+
|user_id|iterations|CHAT|EMAIL|OTHER|PUSH|SMS|
+-------+----------+----+-----+-----+----+---+
|   1022|         5|   2|    0|    1|   0|  2|
|   1004|         4|   1|    1|    1|   1|  0|
|   1013|         4|   0|    0|    2|   1|  1|
|   1020|         4|   2|    0|    1|   1|  0|
```


In [194]:
iterations = df.groupBy("user_id").count().withColumnRenamed("count", "iterations")
iterations.show()

+-------+----------+
|user_id|iterations|
+-------+----------+
|   1016|         1|
|   1005|         4|
|   1031|         2|
|   1030|         5|
|   1034|         3|
|   1008|         1|
|   1026|         2|
|   1028|         1|
|   1029|         2|
|   1032|         3|
|   1010|         3|
|   1035|         1|
|   1037|         3|
|   1036|         2|
|   1022|         1|
|   1041|         1|
|   1001|         3|
|   1020|         2|
|   1006|         5|
|   1007|         2|
+-------+----------+
only showing top 20 rows



In [195]:
# report 2
# TODO
pivot_channel = df.groupBy("user_id").pivot("channel").count().na.fill(0)
pivot_channel.show()


+-------+----+-----+-----+----+---+
|user_id|CHAT|EMAIL|OTHER|PUSH|SMS|
+-------+----+-----+-----+----+---+
|   1025|   2|    0|    0|   0|  0|
|   1016|   1|    0|    0|   0|  0|
|   1005|   0|    0|    4|   0|  0|
|   1031|   1|    0|    1|   0|  0|
|   1034|   1|    1|    1|   0|  0|
|   1030|   1|    0|    1|   2|  1|
|   1046|   0|    1|    0|   0|  0|
|   1008|   0|    1|    0|   0|  0|
|   1021|   0|    0|    0|   1|  0|
|   1026|   0|    1|    0|   1|  0|
|   1028|   0|    0|    0|   0|  1|
|   1032|   1|    1|    0|   0|  1|
|   1029|   1|    0|    0|   0|  1|
|   1010|   2|    1|    0|   0|  0|
|   1002|   0|    0|    0|   0|  1|
|   1035|   0|    0|    0|   1|  0|
|   1045|   0|    0|    0|   0|  1|
|   1037|   1|    0|    1|   1|  0|
|   1036|   0|    0|    0|   1|  1|
|   1022|   0|    0|    0|   1|  0|
+-------+----+-----+-----+----+---+
only showing top 20 rows



In [200]:
iterations.join(pivot_channel, on="user_id", how="left").sort(col("iterations").desc()).show()

+-------+----------+----+-----+-----+----+---+
|user_id|iterations|CHAT|EMAIL|OTHER|PUSH|SMS|
+-------+----------+----+-----+-----+----+---+
|   1030|         5|   1|    0|    1|   2|  1|
|   1006|         5|   0|    3|    0|   2|  0|
|   1005|         4|   0|    0|    4|   0|  0|
|   1038|         4|   0|    1|    0|   2|  1|
|   1014|         4|   3|    1|    0|   0|  0|
|   1033|         4|   0|    1|    0|   3|  0|
|   1018|         4|   2|    0|    1|   1|  0|
|   1034|         3|   1|    1|    1|   0|  0|
|   1032|         3|   1|    1|    0|   0|  1|
|   1010|         3|   2|    1|    0|   0|  0|
|   1037|         3|   1|    0|    1|   1|  0|
|   1001|         3|   0|    2|    0|   0|  1|
|   1003|         3|   0|    1|    1|   1|  0|
|   1031|         2|   1|    0|    1|   0|  0|
|   1026|         2|   0|    1|    0|   1|  0|
|   1029|         2|   1|    0|    0|   0|  1|
|   1036|         2|   0|    0|    0|   1|  1|
|   1020|         2|   0|    0|    0|   2|  0|
|   1007|    

# Challenge 3

In [ ]:
# Theoretical question:

# A new usecase requires the message data to be aggregate in near real time
# They want to build a dashboard embedded in the platform website to analyze message data in low latency (few minutes)
# This application will access directly the data aggregated by streaming process

# Q1:
- What would be your suggestion to achieve that using Spark Structure Streaming?
Or would you choose a different data processing tool?

- Which storage would you use and why? (database?, data lake?, kafka?)

